<a href="https://colab.research.google.com/github/MRamdhan-25/Big-Data/blob/main/242506027_BigData3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),
        ('Markus','Finance', 4200),
        ('Harris', 'Finance', 3500)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
|      Markus|   Finance|  4200|
|      Harris|   Finance|  3500|
+------------+----------+------+



In [50]:
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
|      Markus|  4200|
|      Harris|  3500|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|      Markus|   Finance|  4200|
|      Harris|   Finance|  3500|
+------------+----------+------+

+----------+------------------+
|Department|       avg(Salary)|
+----------+------------------+
|     Sales|            3900.0|
|   Finance|3566.6666666666665|
+----------+------------------+



In [3]:
df_with_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)
df_with_bonus.withColumn('TotalCompensation', df_with_bonus['Salary'] + df_with_bonus['SalaryBonus']).show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
|      Markus|   Finance|  4200|      420.0|           4620.0|
|      Harris|   Finance|  3500|      350.0|           3850.0|
+------------+----------+------+-----------+-----------------+



In [4]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|      Harris|   Finance|  3500|   2|
|      Markus|   Finance|  4200|   3|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+



In [41]:
from google.colab import files
import_files = files.upload()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("BacteriaDataset").getOrCreate()

df = spark.read.csv("/content/bacteria_dataset.csv", header=True, inferSchema=True)

df.printSchema()
df.show(10, truncate=False)

df = df.withColumnRenamed("Name", "Bacteria_Name") \
       .withColumnRenamed("Family", "Bacteria_Family") \
       .withColumnRenamed("Where Found", "Location_Found") \
       .withColumnRenamed("Harmful to Humans", "Is_Harmful")


df = df.withColumn("Kategori",F.when(F.col("Is_Harmful") == "Yes", "Berbahaya")
     .otherwise("Aman"))

count_kategori = df.groupBy("Kategori").count()
count_kategori.show()

count_family = df.groupBy("Bacteria_Family").count().orderBy(F.desc("count"))
count_family.show()

window_family = Window.partitionBy("Bacteria_Family")
df = df.withColumn("Bakteri_Per_Family", F.collect_list("Bacteria_Name").over(window_family))

df.show(10, truncate=False)


Saving bacteria_dataset.csv to bacteria_dataset (9).csv
root
 |-- Name: string (nullable = true)
 |-- Family: string (nullable = true)
 |-- Where Found: string (nullable = true)
 |-- Harmful to Humans: string (nullable = true)

+--------------------------+------------------+-----------------------------+-----------------+
|Name                      |Family            |Where Found                  |Harmful to Humans|
+--------------------------+------------------+-----------------------------+-----------------+
|Escherichia coli          |Enterobacteriaceae|Intestinal tract             |Yes              |
|Staphylococcus aureus     |Staphylococcaceae |Skin, nasal passages         |Yes              |
|Lactobacillus acidophilus |Lactobacillaceae  |Human mouth & intestine      |No               |
|Bacillus subtilis         |Bacillaceae       |Soil                         |No               |
|Clostridium botulinum     |Clostridiaceae    |Soil, improperly canned foods|Yes              |
|Str